# ExaSearchResults

Exa (formerly Metaphor Search) is a search engine fully designed for use by LLMs. Search for documents on the internet using natural language queries, then retrieve cleaned HTML content from desired documents.

Unlike keyword-based search (Google), Exa's neural search capabilities allow it to semantically understand queries and return relevant documents. For example, we could search `"fascinating article about cats"` and compare the search results from Google and Exa. Google gives us SEO-optimized listicles based on the keyword “fascinating”. Exa just works.

This page goes over how to use `ExaSearchResults` with LangChain.

## Overview

### Integration details

| Class | Package | Serializable | [PY support](https://python.langchain.com/docs/integrations/tools/exa_search/) |  Package latest |
| :--- | :--- | :---: | :---: | :---: |
| [ExaSearchResults](https://api.js.langchain.com/classes/langchain_exa.ExaSearchResults.html) | [@langchain/exa](https://npmjs.com/package/@langchain/exa) | ❌ | ✅ |  ![NPM - Version](https://img.shields.io/npm/v/@langchain/exa?style=flat-square&label=%20&) |

## Setup

The integration lives in the `@langchain/exa` package.

```{=mdx}

import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/exa @langchain/core
</Npm2Yarn>

```

### Credentials

First, get an Exa API key and add it as an environment variable. Get 1000 free searches/month by signing up [here](https://dashboard.exa.ai/login).

```typescript
process.env.EXASEARCH_API_KEY="your-api-key"
```

It's also helpful (but not needed) to set up LangSmith for best-in-class observability:

```typescript
process.env.LANGSMITH_TRACING="true"
process.env.LANGSMITH_API_KEY="your-api-key"
```

## Instantiation

Here we show how to insatiate an instance of the `ExaSearchResults` tool:

In [1]:
import { ExaSearchResults } from "@langchain/exa"
import Exa from "exa-js";

// @lc-ts-ignore
const client = new Exa(process.env.EXASEARCH_API_KEY)

const tool = new ExaSearchResults({
  // @lc-ts-ignore
  client,
  searchArgs: {
    numResults: 2,
  }
})

## Invocation

### [Invoke directly with args](/docs/concepts/tools)

In [2]:
await tool.invoke("what is the weather in wailea?")

{"results":[{"score":0.16085544228553772,"title":"Hawaii Weather Forecast","id":"https://www.willyweather.com/hi/hawaii.html","url":"https://www.willyweather.com/hi/hawaii.html","publishedDate":"2023-01-01","author":"","text":"Get an account to remove ads    View More Real-Time Extremes   Nation State County      Hottest 78.8 °FFaleolo Intl / Apia, Samoa, HI    Coldest 51.6 °FBradshaw Army Air Field / Hawaii, HI    Windiest 12.7mphBradshaw Army Air Field / Hawaii, HI    Most Humid 100%Hilo, Hilo International Airport, HI    Least Humid 73.32%Kailua / Kona, Keahole Airport, HI    Highest Pressure 1030.5 hPaBradshaw Army Air Field / Hawaii, HI    Lowest Pressure 1008 hPaFaleolo Intl / Apia, Samoa, HI"},{"score":0.1591680943965912,"title":"The Hawaii Climate To Prepare For Your Maui Wedding","id":"https://mymauiwedding.weebly.com/blog6/the-hawaii-climate-to-prepare-for-your-maui-wedding","url":"https://mymauiwedding.weebly.com/blog6/the-hawaii-climate-to-prepare-for-your-maui-wedding","pu

### [Invoke with ToolCall](/docs/concepts/tools)

We can also invoke the tool with a model-generated `ToolCall`, in which case a `ToolMessage` will be returned:

In [3]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    input: "what is the weather in wailea"
  },
  id: "1",
  name: tool.name,
  type: "tool_call",
}
await tool.invoke(modelGeneratedToolCall)

ToolMessage {
  "content": "{\"results\":[{\"score\":0.12955062091350555,\"title\":\"Urban Dictionary: Waianae\",\"id\":\"https://www.urbandictionary.com/define.php?term=Waianae\",\"url\":\"https://www.urbandictionary.com/define.php?term=Waianae\",\"publishedDate\":\"2006-04-19\",\"author\":\"\",\"text\":\"Hot but good time for go beach ,in this part of Hawaii you HAVE to have respect ,with people and their stuff, but Some people like act dumb and stupid so that’s the only thing that make Waianae look bad , but foreal kine in this part of Hawaii we have respect and if we don’t get that respect you gon expect no respect back .   Get the Waianae mug.    Advertise here for $5/day    Located on the west end of Oahu. Waianae gets a bad reputation for being poor, dirty, scary, etc. Its hot and dry out west and the beaches are super nice. Makaha, Yokes, and Pray for Sex are some great beaches to name a few. Mostly locals and the majority of the homeless live out here. Even though its a little

## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling) and then calling it:

```{=mdx}

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

```

In [4]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
})

In [5]:
import { ChatPromptTemplate } from "@langchain/core/prompts"
import { RunnableConfig } from "@langchain/core/runnables"
import { AIMessage } from "@langchain/core/messages"

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["human", "{user_input}"],
    ["placeholder", "{messages}"],
  ]
)

// specifying tool_choice will force the model to call this tool.
const llmWithTools = llm.bindTools([tool], {
  tool_choice: tool.name
})

const llmChain = prompt.pipe(llmWithTools);

const toolChain = async (userInput: string, config?: RunnableConfig): Promise<AIMessage> => {
  const input_ = { user_input: userInput };
  const aiMsg = await llmChain.invoke(input_, config);
  const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
  return llmChain.invoke({ ...input_, messages: [aiMsg, ...toolMsgs] }, config);
};

const toolChainResult = await toolChain("What is Anthropic's estimated revenue for 2024?");

In [6]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content
}, null, 2))

AIMessage {
  "tool_calls": [
    {
      "name": "exa_search_results_json",
      "args": {
        "input": "Anthropic revenue 2024 projections"
      },
      "type": "tool_call",
      "id": "call_cgC1G9vjXIjHub0TkVfxiDcr"
    }
  ],
  "content": ""
}


## With an Agent

We can create LangChain tools which use the `ExaRetriever` and the `createRetrieverTool` Using these tools we can construct a simple search agent that can answer questions about any topic.

We'll use LangGraph to create the agent. Make sure you have `@langchain/langgraph` installed:

```{=mdx}
<Npm2Yarn>
  @langchain/langgraph
</Npm2Yarn>

Then, define the LLM to use with the agent

<ChatModelTabs customVarName="llmForAgent" />
```

In [7]:
// @lc-docs-hide-cell
import { ChatOpenAI } from "@langchain/openai";

const llmForAgent = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0
})

In [8]:
import Exa from "exa-js";
import { createRetrieverTool } from "langchain/tools/retriever";
import { ExaRetriever } from "@langchain/exa";
import { createReactAgent } from "@langchain/langgraph/prebuilt";

// @lc-ts-ignore
const agentClient = new Exa(process.env.EXASEARCH_API_KEY);

const exaRetrieverForAgent = new ExaRetriever({
  // @lc-ts-ignore
  client: agentClient,
  searchArgs: {
    numResults: 2,
  },
});

// Convert the ExaRetriever into a tool
const searchToolForAgent = createRetrieverTool(exaRetrieverForAgent, {
  name: "search",
  description: "Get the contents of a webpage given a string search query.",
});

const toolsForAgent = [searchToolForAgent];

const agentExecutor = createReactAgent({
  llm: llmForAgent,
  tools: toolsForAgent,
})

In [9]:
const exampleQuery = "Summarize for me a fascinating article about cats."

const events = await agentExecutor.stream(
  { messages: [
    [
      "system",
      `You are a web researcher who answers user questions by looking up information on the internet and retrieving contents of helpful documents. Cite your sources.`,
    ],
    ["human", exampleQuery],
  ] },
  { streamMode: "values", }
)

for await (const event of events) {
  const lastMsg = event.messages[event.messages.length - 1];
  if (lastMsg.tool_calls?.length) {
    console.dir(lastMsg.tool_calls, { depth: null });
  } else if (lastMsg.content) {
    console.log(lastMsg.content);
  }
}

[
  {
    name: 'search',
    args: { query: 'fascinating article about cats' },
    type: 'tool_call',
    id: 'call_EcA0tmWsyNktO7HAsGQnqLVt'
  }
]
No one seems to think brushing kitty's teeth is worth the hassle.      Tyler Comrie / The Atlantic; Getty      On the list of perfect pet parents, Mikel Delgado, a professional feline-behavior consultant, probably ranks high. The Ph.D. expert in animal cognition spends half an hour each evening playing with her three torbie cats, Ruby, Coriander, and Professor Scribbles. She’s trained them to take pills in gelatin capsules, just in case they eventually need meds. She even commissioned a screened-in backyard catio so that the girls can safely venture outside. Delgado would do anything for her cats—well, almost anything. “Guilty as charged,” Delgado told me. “I do not brush my cats’ teeth.” To be fair, most cat owners don’t—probably because they’re well aware that it’s weird, if not downright terrifying, to stick one’s fingers inside an orn

## Related

- Tool [conceptual guide](/docs/concepts/tools)
- Tool [how-to guides](/docs/how_to/#tools)

## API reference

For detailed documentation of all `ExaSearchResults` features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_exa.ExaSearchResults.html)